Importando libraries

In [1]:
import pandas as pd
import numpy as np
import talib
from funcoes_preco import organize_data, get_efficiency_ratio, zigzag
from datetime import datetime, timedelta
from bisect import bisect_right
from scipy import stats

Dados de entrada

In [2]:
# files_dir = 'C:/Users/guicr/AppData/Roaming/MetaQuotes/Terminal/'\
#             'D0E8209F77C8CF37AD8BF550E51FF075/MQL5/Files/'
symbol = 'PETR4'
timeframe = 'M5'
strategy_name = f'mr_zigzag_cci_er_uniquetf_strategy_{symbol}_{timeframe}'
file = f'data/{symbol}_{timeframe}.csv'
# start = datetime(2017, 1, 1).timestamp()
# end = datetime(2020, 12, 31).timestamp()

Gravando dados recebidos em um dicionário

In [3]:
data = organize_data(file)

Enviando os dados para dataframes

In [4]:
symbol_data = data.copy()

Copiando os dataframes para uma variável de manipulação

In [5]:
start = datetime(2021, 12, 21)
end = datetime(2021, 12, 31)
eqty_table = symbol_data.loc[(symbol_data['Date'] >= start) 
            & (symbol_data['Date'] <= end)].copy()
eqty_table.reset_index(inplace=True, drop=True)

Calculando o retorno barra a barra

In [6]:
pct_change = eqty_table['Close'].pct_change()
eqty_table['Percent Change'] = pct_change

Triggers

In [7]:
# O trigger será um CCI calculado dinamicamente com base no comprimento 
# de onda gerado pelo ZigZag. Topo menos Topo e vice-versa.

Filters

In [8]:
# Um filter também será um indicador calculado dinamicamente com base. 
# Este filter é o Efficiency Ratio.

# Cálculo do ZigZag
zigzag_data = zigzag(eqty_table)

In [9]:
eqty_table['ZigZag'] = zigzag_data['zigzag']

Identificando os topos e fundos, a amplitude do movimento e o comprimento
do movimento (Analogias utilizando ondas).

In [10]:
aux_df = pd.DataFrame()
aux_df['zigzag'] = eqty_table['ZigZag'].copy()
aux_df['zigzag'] = aux_df[aux_df['zigzag'] != 0]
aux_df['index'] = aux_df.index
aux_df.dropna(inplace=True)
aux_df_diff = aux_df.diff()

In [12]:
eqty_table['Amplitude'] = np.zeros(len(eqty_table))
eqty_table['Comprimento'] = np.zeros(len(eqty_table))

In [22]:
for row in aux_df_diff.index:
    eqty_table.loc[row, 'Amplitude'] = aux_df_diff.loc[row, 'zigzag']
    eqty_table.loc[row, 'Comprimento'] = aux_df_diff.loc[row, 'index']
    
top_bottom = []
for row in eqty_table.index:
    if eqty_table.loc[row, 'Amplitude'] == 0 or np.isnan(eqty_table.loc[row, 'Amplitude']):
        top_bottom.append(None)
    elif eqty_table.loc[row, 'Amplitude'] < 0:
        top_bottom.append('bottom')
    elif eqty_table.loc[row, 'Amplitude'] > 0:
        top_bottom.append('top')
        
eqty_table['Top_Bottom'] = top_bottom

Cálculo o Efficiency Ratio

In [ ]:
# Cálculo do ER é dinâmico utilizando o comprimento de onda gerado pelo ZigZag.
# Este cálculo é feito alterando dinamicamente o período do ER.
close = eqty_table['Close'].copy()
close_er = get_efficiency_ratio(close)

Preparação do Dataframe para receber as informações de trade.
Integração entre o Timeframe maior com o menor.

Mudar o tipo da coluna 'Date' para datetime.

In [ ]:
eqty_table['Date'] = pd.to_datetime(eqty_table['Date'])

Configuração dos sinais de compra e venda.

In [ ]:
open_position = False
position = []
pos_type = []


shift = 0
er_1 = 0.1
er_2 = 1

d_top = 0
d_bottom = 0

for row in eqty_table[tf_1].index:
    
    if row < cci_period:
        continue
    
    # # Verificação de Topo Duplo ou Fundo Duplo
    # if d_bottom == 0:
    #     if eqty_table[tf_1].loc[row-1, 'CCI'] < -100 and \
    #         eqty_table[tf_1].loc[row, 'CCI'] > -100:
    #             d_bottom += 1
                
    # if d_top == 0:
    #     if eqty_table[tf_1].loc[row-1, 'CCI'] > 100 and \
    #         eqty_table[tf_1].loc[row, 'CCI'] < 100:
    #             d_top += 1
                
    # if d_bottom > 0 and eqty_table[tf_1].loc[row, 'CCI'] > 0:
    #     d_bottom = 0
    
    # if d_top > 0 and eqty_table[tf_1].loc[row, 'CCI'] < 0:
    #     d_top = 0                

    # Sinal entrada de Compra
    elif not open_position and \
        d_bottom == 0 and \
        eqty_table[tf_1].loc[row-1, 'CCI'] < 0 and \
        eqty_table[tf_1].loc[row, 'CCI'] > 0 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-shift, 
                                'Skewness'] < -0.5 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-shift, 
                                'Skewness'] > -1 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-shift,
                              'Efficiency Ratio'] < er_1 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-1-shift,
                              'Efficiency Ratio'] < er_2:
            open_position = True
            pos_type = 'Buy'
            # Revisar como fazer arredondamento de 5 em 5
            stop_loss = eqty_table[tf_1].loc[row, 'Close'] - \
                (5 * np.ceil((2 * eqty_table[tf_1].loc[row, 'Standard Deviation']) / 5))
            std = eqty_table[tf_1].loc[row, 'Standard Deviation']
            position.append([row, pos_type, eqty_table[tf_1].loc[row+1, 'Open'],
                             std, stop_loss])
    
    # Sinal de Saída da Compra
    elif open_position and \
        pos_type == 'Buy' and \
        eqty_table[tf_1].loc[row-1, 'CCI'] < 100 and \
        eqty_table[tf_1].loc[row, 'CCI'] > 100:
            open_position = False
            pos_type = 'Exit Buy'
            position.append([row, pos_type, eqty_table[tf_1].loc[row, 'Close'],
                             np.nan, np.nan])
    
    elif open_position and \
        pos_type == 'Buy' and \
        eqty_table[tf_1].loc[row-1, 'CCI'] > -100 and \
        eqty_table[tf_1].loc[row, 'CCI'] < -100:
            open_position = False
            pos_type = 'Exit Buy'
            position.append([row, pos_type, eqty_table[tf_1].loc[row, 'Close'],
                             np.nan, np.nan])
    
    elif open_position and \
        eqty_table[tf_1].loc[row, 'Low'] <= stop_loss:
            open_position = False
            pos_type = 'Exit Buy'
            position.append([row, pos_type, stop_loss, np.nan, np.nan])
            
    # Sinal entrada de Venda
    elif not open_position and \
        d_top == 0 and \
        eqty_table[tf_1].loc[row-1, 'CCI'] > 0 and \
        eqty_table[tf_1].loc[row, 'CCI'] < 0 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-shift, 
                                'Skewness'] > 0.5 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-shift, 
                                'Skewness'] < 1 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-shift,
                              'Efficiency Ratio'] < er_1 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-1-shift,
                              'Efficiency Ratio'] < er_2:
            open_position = True
            pos_type = 'Sell'
            stop_loss = eqty_table[tf_1].loc[row, 'Close'] + \
                (5 * np.ceil((2 * eqty_table[tf_1].loc[row, 'Standard Deviation']) / 5))
            std = eqty_table[tf_1].loc[row, 'Standard Deviation']
            position.append([row, pos_type, eqty_table[tf_1].loc[row+1, 'Open'],
                             std, stop_loss])
            
    # Sinal de Saída da Venda
    elif open_position and \
        pos_type == 'Sell' and \
        eqty_table[tf_1].loc[row-1, 'CCI'] > -100 and \
        eqty_table[tf_1].loc[row, 'CCI'] < -100:
            open_position = False
            pos_type = 'Exit Sell'
            position.append([row, pos_type, eqty_table[tf_1].loc[row, 'Close'],
                             np.nan, np.nan])
    
    elif open_position and \
        pos_type == 'Sell' and \
        eqty_table[tf_1].loc[row-1, 'CCI'] < 100 and \
        eqty_table[tf_1].loc[row, 'CCI'] > 100:
            open_position = False
            pos_type = 'Exit Sell'
            position.append([row, pos_type, eqty_table[tf_1].loc[row, 'Close'],
                             np.nan, np.nan])
            
    elif open_position and \
        eqty_table[tf_1].loc[row, 'High'] >= stop_loss:
            open_position = False
            pos_type = 'Exit Sell'
            position.append([row, pos_type, stop_loss, np.nan, np.nan])
    
    # Cálculo da Máxima Exposição Favorável e Máxima Exposição Desfavorável
    

In [ ]:
eqty_table[tf_1]

Cálculo do retorno.

In [ ]:
df = pd.DataFrame(position, columns=['Index', 'Position Type', 'Position', 
                                     'Standard Deviation', 'Stop Loss'])

In [ ]:
df

In [ ]:
df2 = df.iloc[::2] 
df2.reset_index(inplace=True, drop=True)

In [ ]:
df3 = df.iloc[1::2]
df3.reset_index(inplace=True, drop=True)
df3.rename(columns={'Position': 'Position Exit', 
                    'Position Type': 'Position Type Exit',
                    'Standard Deviation': 'Standard Deviation Exit',
                    'Stop Loss': 'Stop Loss Exit'}, inplace=True)

In [ ]:
df = pd.concat([df2, df3], axis=1)
df

In [ ]:
points = []
for row in df.index:
    if df.loc[row, 'Position Type'] == 'Buy':
        points = np.append(points, df.loc[row, 'Position Exit'] - df.loc[row, 'Position'])
    else:
        points = np.append(points, [df.loc[row, 'Position'] - df.loc[row, 'Position Exit']])

In [ ]:
stds = df['Standard Deviation']

In [ ]:
r = points / stds

In [ ]:
r.dropna(inplace=True)

In [ ]:
r

Estatísticas do retorno

In [ ]:
last_value = 0
win_stk = 0
loss_stk = 0
pos_stk = 0
neg_stk = 0
max_sum_r = 0
sum_ant = 0
dd = []
for value in r:
    if value >= 0 and last_value >= 0:
        win_stk += 1
        if win_stk > pos_stk:
            pos_stk = win_stk
    elif value < 0 and last_value <= 0:
        loss_stk += 1
        if loss_stk > neg_stk:
            neg_stk = loss_stk
    else:
        win_stk = 1
        loss_stk = 1
    
    last_value = value
    
    sum_r = sum_ant + value
    sum_ant = sum_r
    if max_sum_r < sum_r:
        max_sum_r = sum_r
    dd.append(sum_r - max_sum_r)
    
print(pos_stk)
print(neg_stk)
print(abs(min(dd)))   

In [ ]:
sum(r)

In [ ]:
stats.describe(r)

In [ ]:
z_scores = stats.zscore(r)
abs_z_scores = np.abs(z_scores)
filtered_r = r[abs_z_scores < 3]
stats.describe(filtered_r)

In [ ]:
sum(filtered_r)

In [ ]:
strategy_data = {
    'tf_1': tf_1,
    'tf_2': tf_2,
    'cci_period': cci_period,
    'std_dev_period': std_dev_period,
    'er_period': er_period,
    'skew_period': skew_period,
    'shift': shift,
    'er_1': er_1,
    'er_2': er_2,
}
filename = f'{strategy_name}_specs.txt'
with open(filename, 'w') as f:
    f.write(str(strategy_data))